In [1]:
test_input = '''seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
'''
test_output_1 = 35
test_output_2 = 46

In [2]:
from collections import defaultdict
from dataclasses import dataclass
from functools import cached_property

In [3]:
def check_mappings(n, mappings):
    for m in mappings:
        if dst:= m.src_to_dst(n):
            return dst
    return n

def reverse_mappings(n, mappings):
    for m in mappings:
        if src:=m.dst_to_src(n):
            return src
    return n

In [4]:
@dataclass
class Mapping:
    dst_start: int
    src_start: int
    range_len: int

    def src_to_dst(self, src):
        if self.src_start+self.range_len > src >= self.src_start:
            return src-self.src_start + self.dst_start
        return False

    def dst_to_src(self, dst):
        if self.dst_start <= dst < self.dst_start + self.range_len:
            return dst - self.dst_start + self.src_start
        return False

In [5]:
@dataclass
class SeedRange:
    start: int
    range_len: int

    def in_range(self, n):
        return self.start <= n < self.start + self.range_len

In [6]:
@dataclass
class Seed:
    seed: int
    soil: int = -1
    fert: int = -1
    watr: int = -1
    lght: int = -1
    temp: int = -1
    humi: int = -1
    loca: int = -1

    def fillout(self, mappings):
        self.soil = check_mappings(self.seed, mappings['soil'])
        self.fert = check_mappings(self.soil, mappings['fertilizer'])
        self.watr = check_mappings(self.fert, mappings['water'])
        self.lght = check_mappings(self.watr, mappings['light'])
        self.temp = check_mappings(self.lght, mappings['temperature'])
        self.humi = check_mappings(self.temp, mappings['humidity'])
        self.loca = check_mappings(self.humi, mappings['location'])

    @classmethod
    def from_loc(cls, mappings, loc):
        hum = reverse_mappings(loc, mappings['location'])
        tem = reverse_mappings(hum, mappings['humidity'])
        lig = reverse_mappings(tem, mappings['temperature'])
        wat = reverse_mappings(lig, mappings['light'])
        fer = reverse_mappings(wat, mappings['water'])
        soi = reverse_mappings(fer, mappings['fertilizer'])
        see = reverse_mappings(soi, mappings['soil'])
        return cls(see, soi, fer, wat, lig, tem, hum, loc)

In [7]:
def parse_almanac(text):
    chapters = text.strip().split('\n\n')
    seeds1 = []
    mappings = defaultdict(list)
    seed_ranges = []
    for ch in chapters:
        chname, chnums = (x.strip() for x in ch.split(':'))
        if chname == 'seeds':
            seed_nums = chnums.strip().split(' ')
            seeds1 = [Seed(int(n)) for n in seed_nums]
            #seeds2 = {n for i in range(0, len(seed_nums), 2) for n in range(int(seed_nums[i]), int(seed_nums[i])+int(seed_nums[i+1]))}
            seed_ranges = [SeedRange(int(seed_nums[i]), int(seed_nums[i+1])) for i in range(0, len(seed_nums), 2)]
            continue
        chname = chname.replace(' map', '').split('-to-')[1]
        for row in chnums.strip().split('\n'):
            mappings[chname].append(Mapping(*[int(n) for n in row.strip().split(' ')]))
    return seeds1, dict(mappings), seed_ranges
#parse_almanac(test_input)

In [8]:
def solve1(text):
    seeds, mappings, _ = parse_almanac(text)
    for seed in seeds:
        seed.fillout(mappings)
    return sorted(seeds, key=lambda x:x.loca)[0].loca
assert solve1(test_input) == test_output_1

In [10]:
def solve2(text):
    _, mappings, seed_ranges = parse_almanac(text)
    loc = 0
    while loc < 3_000_000_000:
        loc += 1
        seed = Seed.from_loc(mappings, loc)
        for sr in seed_ranges:
            if sr.in_range(seed.seed):
                return loc
    #_, mappings, seeds = parse_almanac(text)
    #min_loc = float('inf')
    #for seed in seeds:
    #    seed.fillout(mappings)
    #    min_loc = min(min_loc, seed.loca)
    #return min_loc
assert solve2(test_input) == test_output_2

In [11]:
with open('day05.txt') as FILE:
    print(solve1(FILE.read()))

318728750


In [12]:
with open('day05.txt') as FILE:
    print(solve2(FILE.read()))

37384986
